## The Portfolio Class

A number of the portfolio webpages make use of the Portfolio class object defined below.  The Portfolio class object implements the various quadratic programming problems encountered in mean-variance analysis using the CVXOPT package.  The CVXOPT user guide is available <a href="https://cvxopt.org/userguide/index.html"> here </a>.

The function `cvxopt.solvers.qp` solves problems of the general form:
\begin{align*}
    \underset{w}{\text{min  }}& \frac{1}{2} w' Q w + p'x \\
     \text{subject to  } & Gw \le h \\
                        & Aw = b \\
\end{align*}

For our purposes, the optimizer is usually solving for a $N \times 1$ vector of portfolio weights, $w$.  The objective function often takes $Q$ as the $N \times N$ covariance matrix of returns and $p$ as a $N \times 1$ vector of zeros.  

The constraint $Gw \le h$ can be used to put position limits on the portfolio weights.  The most common use of this is short-selling constraints.  This is accomplished by setting $G$ equal to the negative of the $N \times N$ identity matrix (that is, a matrix with -1s on the diagonal and zero elsewhere), and $h$ equal to a $N \times 1$ vector of zeros.  Additional position limits could be enforced by adding rows to $G$ and $h$.  For instance, many funds have maximum position limits in addition to short-sale constraints.

The constraint $Aw=b$ usually includes the constraint that the portfolio is fulling invested; that is, the portfolio weights sum to 100%.  This is accomplished by setting $A$ equal to a $N \times 1$ vector of ones and setting $b=1$.  $A$ and $b$ can include additional rows for other constraints.  For instance, setting a target expected return would be accomplished by adding a row to $A$ with the asset mean vector and adding a row to $b$ with the target expected return.  This is done when solving for a point on the mean-variance frontier.


``` p
import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar

SolverOptions["show_progress"] = False

class portfolio:
    def __init__(self, means, cov, Shorts):
        self.means = np.array(means)
        self.cov = np.array(cov)
        self.Shorts = Shorts
        self.n = len(means)
        if Shorts:
            w = np.linalg.solve(cov, np.ones(self.n))
            self.GMV = w / np.sum(w)
            w = np.linalg.solve(cov, means)
            self.piMu = w / np.sum(w)
        else:
            n = self.n
            Q = matrix(cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.ones(n), (1, n), tc="d")
            b = matrix([1], (1, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            self.GMV = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def frontier(self, m):
        if self.Shorts:
            gmv = self.GMV
            piMu = self.piMu
            m1 = gmv @ self.means
            m2 = piMu @ self.means
            a = (m - m2) / (m1 - m2)
            return a * gmv + (1 - a) * piMu
        else:
            n = self.n
            Q = matrix(self.cov, tc="d")
            p = matrix(np.zeros(n), (n, 1), tc="d")
            G = matrix(-np.identity(n), tc="d")
            h = matrix(np.zeros(n), (n, 1), tc="d")
            A = matrix(np.vstack((np.ones(n), self.means)), (2, n), tc="d")
            b = matrix([1, m], (2, 1), tc="d")
            sol = Solver(Q, p, G, h, A, b)
            return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])

    def tangency(self, r):
        if self.Shorts:
            w = np.linalg.solve(self.cov, self.means - r)
            return w / np.sum(w)
        else:
            def f(m):
                w = self.frontier(m)
                mn = w @ self.means
                sd = np.sqrt(w.T @ self.cov @ w)
                return - (mn - r) / sd
            m = minimize_scalar(f, bounds=[max(r, np.min(self.means)), max(r, np.max(self.means))], method="bounded").x
            return self.frontier(m)

    def optimal(self, raver, rs=None, rb=None):
        n = self.n
        if self.Shorts:
            if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n + 2, 1), tc="d")
                G = np.zeros((2, n + 2))
                G[0, 0] = G[1, 1] = -1
                G = matrix(G, (2, n+2), tc="d")
                h = matrix([0, 0], (2, 1), tc="d")
                A = matrix([1, -1] + n*[1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
            else:
                w = np.linalg.solve(self.cov, self.means)
                a = np.sum(w)
                return (a/raver)*self.piMu + (1-a/raver)*self.GMV
        else:
           if (rs or rs==0) and (rb or rb==0):
                Q = np.zeros((n + 2, n + 2))
                Q[2:, 2:] = raver * self.cov
                Q = matrix(Q, tc="d")
                p = np.array([-rs, rb] + list(-self.means))
                p = matrix(p, (n+2, 1), tc="d")
                G = matrix(-np.identity(n + 2), tc="d")
                h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
                A = matrix([1, -1] + n * [1], (1, n+2), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten()[2:] if sol["status"] == "optimal" else None
           else:
                Q = matrix(raver * self.cov, tc="d")
                p = matrix(-self.means, (n, 1), tc="d")
                G = matrix(-np.identity(n), tc="d")
                h = matrix(np.zeros(n), (n, 1), tc="d")
                A = matrix(np.ones(n), (1, n), tc="d")
                b = matrix([1], (1, 1), tc="d")
                sol = Solver(Q, p, G, h, A, b)
                return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else None
```

## Understanding the Portfolio Class

In [4]:

import numpy as np
from cvxopt import matrix
from cvxopt.solvers import qp as Solver, options as SolverOptions
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize
##### Inputs
# Risk-free rate
r = 0.02
# Expected returns
means = np.array([0.06, 0.065, 0.08])
# Standard deviations
sds = np.array([0.15, 0.165, 0.21])
# Correlations
corr12 = 0.75
corr13 = 0.75
corr23 = 0.75
# Covariance matrix
C  = np.identity(3)
C[0, 1] = C[1, 0] = corr12
C[0, 2] = C[2, 0] = corr13
C[1, 2] = C[2, 1] = corr23
cov = np.diag(sds) @ C @ np.diag(sds)
Shorts = False



# No Short-selling


The GMV problem:
$$ \underset{w_1,w_2,\dots,w_N}{\text{min}} \text{var}[r_p]$$ 
subject to the constraints $\sum_i w_i=1$ and $w_i \ge 0$ for all assets $i$


In [5]:
# The GMV problem with short-sale constraints
n = len(means)
Q = matrix(cov, tc="d")
p = matrix(np.zeros(n), (n, 1), tc="d")
# Constraint: short-sales not allowed
G = matrix(-np.identity(n), tc="d")
h = matrix(np.zeros(n), (n, 1), tc="d")
# Constraint: fully-invested portfolio
A = matrix(np.ones(n), (1, n), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
wgts_gmv = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
print('GMV portfolio is: ', wgts_gmv)

     pcost       dcost       gap    pres   dres
 0:  1.2737e-02 -9.9119e-01  1e+00  0e+00  2e+00
 1:  1.2705e-02 -1.1636e-03  1e-02  1e-16  3e-02
 2:  1.1613e-02  9.6474e-03  2e-03  1e-16  4e-03
 3:  1.0788e-02  1.0529e-02  3e-04  1e-16  3e-18
 4:  1.0644e-02  1.0634e-02  1e-05  6e-17  2e-18
 5:  1.0635e-02  1.0635e-02  1e-07  2e-16  4e-18
 6:  1.0635e-02  1.0635e-02  1e-09  1e-16  8e-18
Optimal solution found.
GMV portfolio is:  [6.87499842e-01 3.12499846e-01 3.11399161e-07]


The frontier problem:
$$ \underset{w_1,w_2,\dots,w_N}{\text{min}} \text{var}[r_p]$$ 
subject to the constraints $\sum_i w_i E[r_i]=E[r_{\text{target}}]$, $\sum_i w_i=1$ and $w_i \ge 0$ for all assets $i$.


In [6]:

# The frontier problem with short-sale constraints for target return m
m = 0.07        # a target expected return
n = len(means)
Q = matrix(cov, tc="d")
p = matrix(np.zeros(n), (n, 1), tc="d")
G = matrix(-np.identity(n), tc="d")
h = matrix(np.zeros(n), (n, 1), tc="d")
A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
b = matrix([1, m], (2, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
wgts_frontier = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
print('Frontier portfolio for expected return of ' + f'{m:.1%}' + ' is: ', wgts_frontier)


     pcost       dcost       gap    pres   dres
 0:  1.3663e-02 -1.0632e+00  1e+00  7e-18  2e+00
 1:  1.3663e-02  2.7786e-03  1e-02  1e-16  2e-02
 2:  1.3662e-02  1.3470e-02  2e-04  6e-17  4e-04
 3:  1.3661e-02  1.3658e-02  3e-06  1e-16  5e-06
 4:  1.3661e-02  1.3661e-02  3e-08  2e-16  5e-08
Optimal solution found.
Frontier portfolio for expected return of 7.0% is:  [0.25461946 0.32717405 0.41820649]



The tangency problem:
$$ \underset{w_1,w_2,\dots,w_N}{\text{max}} \frac{E[r_p] - r_f}{\text{sd}[r_p]} $$ 
subject to the constraints $\sum_i w_i=1$ and $w_i \ge 0$ for all assets $i$.


In [7]:

# Method 1: Tangency problem with short-sale constraints
# The approach here is to find the expected return of the frontier portfolio with the highest sharpe ratio
def frontier(m):
    Q = matrix(cov, tc="d")
    p = matrix(np.zeros(n), (n, 1), tc="d")
    G = matrix(-np.identity(n), tc="d")
    h = matrix(np.zeros(n), (n, 1), tc="d")
    A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
    b = matrix([1, m], (2, 1), tc="d")
    sol = Solver(Q, p, G, h, A, b)
    return np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
def f(m):
    w = frontier(m)
    mn = w @ means
    sd = np.sqrt(w.T @ cov @ w)
    return -(mn - r) / sd
min_expret = max(r, np.min(means))
max_expret = max(r, np.max(means))
m = minimize_scalar(f, bounds=[min_expret,max_expret], method="bounded").x
wgts_tangency = frontier(m)
print('Tangency portfolio is: ', wgts_tangency)

     pcost       dcost       gap    pres   dres
 0:  1.2463e-02 -1.0120e+00  1e+00  6e-17  2e+00
 1:  1.2463e-02  2.2152e-03  1e-02  6e-17  2e-02
 2:  1.2463e-02  1.2358e-02  1e-04  1e-16  2e-04
 3:  1.2463e-02  1.2462e-02  1e-06  6e-17  2e-06
 4:  1.2463e-02  1.2463e-02  1e-08  1e-16  2e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.5134e-02 -1.1268e+00  1e+00  7e-16  3e+00
 1:  1.5134e-02  3.5240e-03  1e-02  1e-16  3e-02
 2:  1.5131e-02  1.4866e-02  3e-04  2e-16  6e-04
 3:  1.5125e-02  1.5119e-02  6e-06  1e-16  8e-06
 4:  1.5125e-02  1.5125e-02  6e-08  1e-16  8e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  1.1354e-02 -1.0675e+00  1e+00  8e-17  3e+00
 1:  1.1354e-02  4.5312e-04  1e-02  4e-16  3e-02
 2:  1.1352e-02  1.1170e-02  2e-04  1e-16  4e-04
 3:  1.1351e-02  1.1348e-02  2e-06  1e-16  5e-06
 4:  1.1351e-02  1.1351e-02  2e-08  3e-17  5e-08
Optimal solution found.
     pcost       dcost       gap    pres   dres
 

In [8]:
# Method 2: Tangency problem with short-sale constraints
# The approach here is to directly minimize the negative Sharpe ratio by changing the vector of weights
n = len(means)
def f(w):
    mn = w @ means
    sd = np.sqrt(w.T @ cov @ w)
    return -(mn - r) / sd
# Initial guess (equal-weighted)
w0 = (1/n)*np.ones(n)
# Constraint: fully-invested portfolio
A = np.ones(n)
b = 1
cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
# Short-sale constraint
bnds = [(0, None) for i in range(n)]   #Short-sale constraint
# Optimization
wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
print('Tangency portfolio is: ', wgts_tangency)

Tangency portfolio is:  [0.27515622 0.32986942 0.39497436]


### Optimal portfolios based on risk aversion:

#### With risk-free rate(s)

The optimal portfolio for investor with risk aversion $A$:
$$ \underset{w_{\text{saving}},w_{\text{borrow}},w_1,w_2,\dots,w_N}{\text{max}} E[r_p] - 0.5 \cdot A \text{var}[r_p] $$ 
subject to the constraints $w_{\text{saving}} - w_{\text{borrow}}+\sum_i w_i=1$, $w_i \ge 0$ for all assets $i=1,...,N$, $w_{\text{saving}} \ge 0$, and $w_{\text{borrow}} \ge 0$.  Under this notation where the borrowing weight is positive, the return on borrowing is negative: $-r_b < 0$.

Alternatively, we can use the usual notation where borrowing is represented by $w_{\text{borrow}}<0$.  In this case, the constraints are $w_{\text{saving}} + w_{\text{borrow}}+\sum_i w_i=1$, $w_i \ge 0$ for all assets $i=1,...,N$, $w_{\text{saving}} \ge 0$, and $w_{\text{borrow}} \le 0$, and the return on the borrowing asset is $r_b>0$.


In [16]:
# Case #1: multiple borrowing rates
raver = 3

# Risk-free saving
rs = 0.02
# Risk-free borrowing
rb = 0.04

In [17]:
# Convention that borrowing weight is positive (this is what is coded in portfolio class)
Q = np.zeros((n + 2, n + 2))
Q[2:, 2:] = raver * cov
Q = matrix(Q, tc="d")
p = np.array([-rs, rb] + list(-means))
p = matrix(p, (n + 2, 1), tc="d")
# Constraint: short-sales not allowed, saving weight positive, borrowing weight positive
G = matrix(-np.identity(n + 2), tc="d")
h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
# Constraint: fully-invested portfolio
A = matrix([1, -1] + n*[1], (1, n+2), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
if sol["status"] == "optimal":
    wgts_optimal = np.array(sol["x"]).flatten()[2:]
else:
    wgts_optimal = None
print('Optimal risky portfolio for investor with risk aversion of ' + f'{raver:.1f}' + ' is: ', wgts_optimal)
print('Saving weight: ' + f'{np.array(sol["x"]).flatten()[0]: .1%}')
print('Borrow weight: ' + f'{np.array(sol["x"]).flatten()[1]: .1%}')
print('Total weight in risky assets is: ' + f'{np.sum(wgts_optimal): .1%}')

     pcost       dcost       gap    pres   dres
 0: -3.9456e-02 -5.2426e-01  7e+00  3e+00  3e+00
 1:  3.7880e-02 -9.7271e-01  1e+00  4e-01  4e-01
 2:  1.4246e-02 -1.1588e-01  1e-01  8e-16  9e-17
 3: -3.3521e-02 -6.1566e-02  3e-02  3e-16  6e-17
 4: -2.6937e-02 -5.0459e-02  2e-02  2e-16  3e-17
 5: -3.5144e-02 -3.6520e-02  1e-03  1e-16  6e-18
 6: -3.5252e-02 -3.5314e-02  6e-05  9e-17  1e-17
 7: -3.5255e-02 -3.5256e-02  7e-07  6e-17  5e-18
 8: -3.5255e-02 -3.5255e-02  7e-09  1e-16  1e-17
Optimal solution found.
Optimal risky portfolio for investor with risk aversion of 3.0 is:  [0.17008277 0.20359422 0.24242381]
Saving weight:  38.4%
Borrow weight:  0.0%
Total weight in risky assets is:  61.6%


In [18]:
# Convention where borrowing portfolio weight is negative
Q = np.zeros((n + 2, n + 2))
Q[2:, 2:] = raver * cov
Q = matrix(Q, tc="d")
p = np.array([-rs, -rb] + list(-means))
p = matrix(p, (n + 2, 1), tc="d")
# Constraint: short-sales not allowed, saving weight positive, borrowing weight negative
G = -np.identity(n + 2)
G[1, 1] = 1
G = matrix(G, (n+2, n+2), tc="d")
h = matrix(np.zeros(n+2), (n+2, 1), tc="d")
# Constraint: fully-invested portfolio
A = matrix(np.ones(n+2), (1, n+2), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
if sol["status"] == "optimal":
    wgts_optimal = np.array(sol["x"]).flatten()[2:]
else:
    wgts_optimal = None
print('Optimal risky portfolio for investor with risk aversion of ' + f'{raver:.1f}' + ' is: ', wgts_optimal)
print('Saving weight: ' + f'{np.array(sol["x"]).flatten()[0]: .1%}')
print('Borrow weight: ' + f'{np.array(sol["x"]).flatten()[1]: .1%}')
print('Total weight in risky assets is: ' + f'{np.sum(wgts_optimal): .1%}')

     pcost       dcost       gap    pres   dres
 0: -3.9456e-02 -5.2426e-01  7e+00  3e+00  3e+00
 1:  3.7880e-02 -9.7271e-01  1e+00  4e-01  4e-01
 2:  1.4246e-02 -1.1588e-01  1e-01  8e-16  9e-17
 3: -3.3521e-02 -6.1566e-02  3e-02  3e-16  6e-17
 4: -2.6937e-02 -5.0459e-02  2e-02  2e-16  3e-17
 5: -3.5144e-02 -3.6520e-02  1e-03  1e-16  6e-18
 6: -3.5252e-02 -3.5314e-02  6e-05  9e-17  1e-17
 7: -3.5255e-02 -3.5256e-02  7e-07  6e-17  5e-18
 8: -3.5255e-02 -3.5255e-02  7e-09  1e-16  1e-17
Optimal solution found.
Optimal risky portfolio for investor with risk aversion of 3.0 is:  [0.17008277 0.20359422 0.24242381]
Saving weight:  38.4%
Borrow weight: -0.0%
Total weight in risky assets is:  61.6%


#### Risky-assets only (without risk-free assets)


In [19]:
# Convention where borrowing portfolio weight is negative
Q = matrix(raver * cov, tc="d")
p = matrix(-means, (n, 1), tc="d")
# Constraint: short-sales not allowed, saving weight positive, borrowing weight negative
G = matrix(-np.identity(n), tc="d")
h = matrix(np.zeros(n), (n, 1), tc="d")
# Constraint: fully-invested portfolio
A = matrix(np.ones(n), (1, n), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
if sol["status"] == "optimal":
    wgts_optimal = np.array(sol["x"]).flatten()
else:
    wgts_optimal = None
print('Optimal risky portfolio for investor with risk aversion of ' + f'{raver:.1f}' + ' is: ', wgts_optimal)

     pcost       dcost       gap    pres   dres
 0: -3.0013e-02 -1.0336e+00  1e+00  0e+00  2e+00
 1: -3.0040e-02 -4.3508e-02  1e-02  1e-16  3e-02
 2: -3.0567e-02 -3.1779e-02  1e-03  1e-16  2e-03
 3: -3.0682e-02 -3.0757e-02  7e-05  1e-16  3e-05
 4: -3.0684e-02 -3.0685e-02  9e-07  8e-17  3e-07
 5: -3.0684e-02 -3.0684e-02  9e-09  8e-17  3e-09
Optimal solution found.
Optimal risky portfolio for investor with risk aversion of 3.0 is:  [0.47066932 0.36021959 0.16911109]


# With Short-Sales
The GMV problem:
$$ \underset{w_1,w_2,\dots,w_N}{\text{min}} \text{var}[r_p]$$ 
subject to the constraints $\sum_i w_i=1$


In [20]:
# Method 1: GMV: theoretical solution
n = len(means)
w = np.linalg.solve(cov, np.ones(n))
wgts_gmv = w / np.sum(w)
print('GMV portfolio is: ', wgts_gmv)
# Method 2: The GMV problem without short-sale constraints
n = len(means)
Q = matrix(cov, tc="d")
p = matrix(np.zeros(n), (n, 1), tc="d")
# No position limits
G = matrix(np.zeros((n,n)), tc="d")
h = matrix(np.zeros(n), (n, 1), tc="d")
# Fully-invested constraint
A = matrix(np.ones(n), (1, n), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
wgts_gmv = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
print('GMV portfolio is: ', wgts_gmv)

GMV portfolio is:  [ 0.78298611  0.40798611 -0.19097222]
     pcost       dcost       gap    pres   dres
 0:  1.0339e-02  1.0339e-02  3e+00  2e+00  0e+00
 1:  1.0339e-02  1.0339e-02  3e-02  2e-02  0e+00
 2:  1.0339e-02  1.0339e-02  3e-04  2e-04  0e+00
 3:  1.0339e-02  1.0339e-02  3e-06  2e-06  0e+00
 4:  1.0339e-02  1.0339e-02  3e-08  2e-08  0e+00
Optimal solution found.
GMV portfolio is:  [ 0.78298611  0.40798611 -0.19097222]


The frontier problem:
$$ \underset{w_1,w_2,\dots,w_N}{\text{min}} \text{var}[r_p]$$ 
subject to the constraints $\sum_i w_i E[r_i]=E[r_{\text{target}}]$, $\sum_i w_i=1$.


In [21]:

# Method 1: frontier: theoretical solution without short-sale constraint
# m=target expected return
m = 0.07
n = len(means)
w = np.linalg.solve(cov, np.ones(n))
wgts_gmv = w / np.sum(w)
w = np.linalg.solve(cov, means)
piMu = w / np.sum(w)
m1 = wgts_gmv @ means
m2 = piMu @ means
a = (m - m2) / (m1 - m2)
wgts_frontier= a * wgts_gmv + (1 - a) * piMu

print('Frontier portfolio for expected return of ' + f'{m:.1%}' + ' is: ', wgts_frontier)


Frontier portfolio for expected return of 7.0% is:  [0.25461741 0.32717678 0.4182058 ]


In [22]:
# Method 2: frontier: numerical solutions without short-sale constraint
# m=target expected return
m = 0.07
n = len(means)
Q = matrix(cov, tc="d")
p = matrix(np.zeros(n), (n, 1), tc="d")
# No position limits
G = matrix(np.zeros((n,n)), tc="d")
h = matrix(np.zeros(n), (n, 1), tc="d")
# Fully-invested constraint
A = matrix(np.vstack((np.ones(n), means)), (2, n), tc="d")
b = matrix([1, m], (2, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
wgts_frontier = np.array(sol["x"]).flatten() if sol["status"] == "optimal" else np.array(n * [np.nan])
print('Frontier portfolio for expected return of ' + f'{m:.1%}' + ' is: ', wgts_frontier)


     pcost       dcost       gap    pres   dres
 0:  1.3661e-02  1.3661e-02  3e+00  2e+00  8e-18
 1:  1.3661e-02  1.3661e-02  3e-02  2e-02  1e-17
 2:  1.3661e-02  1.3661e-02  3e-04  2e-04  3e-18
 3:  1.3661e-02  1.3661e-02  3e-06  2e-06  8e-18
 4:  1.3661e-02  1.3661e-02  3e-08  2e-08  0e+00
Optimal solution found.
Frontier portfolio for expected return of 7.0% is:  [0.25461741 0.32717678 0.4182058 ]


The tangency problem:
$$ \underset{w_1,w_2,\dots,w_N}{\text{max}} \frac{E[r_p] - r_f}{\text{sd}[r_p]} $$ 
subject to the constraints $\sum_i w_i=1$.


In [23]:
# Method 1: tangency: theoretical solution without short-sale constraint
w = np.linalg.solve(cov, means - r)
wgts_tangency = w / np.sum(w)
print('Tangency portfolio is: ', wgts_tangency)

Tangency portfolio is:  [0.27606178 0.33045651 0.39348172]


In [24]:
# Method #2: tangency: numerical solution without short-sale constraints
# Directly minimize the negative Sharpe ratio by changing the weights
n = len(means)
def f(w):
    mn = w @ means
    sd = np.sqrt(w.T @ cov @ w)
    return -(mn - r) / sd
# Initial guess (equal-weighted)
w0 = (1/n)*np.ones(n)
# Constraint: fully-invested portfolio
A = np.ones(n)
b = 1
cons = [{"type": "eq", "fun": lambda x: A @ x - b}]
# No short-sale constraint
bnds = [(None, None) for i in range(n)] 
# Optimization
wgts_tangency = minimize(f, w0, bounds=bnds, constraints=cons).x
print('Tangency portfolio is: ', wgts_tangency)

Tangency portfolio is:  [0.27515622 0.32986942 0.39497436]


### Optimal portfolios based on risk aversion:


The optimal portfolio for investor with risk aversion $A$:
$$ \underset{w_{\text{saving}},w_{\text{borrow}},w_1,w_2,\dots,w_N}{\text{max}} E[r_p] - 0.5 \cdot A \text{var}[r_p] $$ 
subject to the constraints $w_{\text{saving}} - w_{\text{borrow}}+\sum_i w_i=1$, $w_{\text{saving}} \ge 0$, and $w_{\text{borrow}} \ge 0$.  Under this notation where the borrowing weight is positive, the return on borrowing is negative: $-r_b < 0$. 

Alternatively, we can use the usual notation where borrowing is represented by $w_{\text{borrow}}<0$.  In this case, the constraints are $w_{\text{saving}} + w_{\text{borrow}}+\sum_i w_i=1$, $w_{\text{saving}} \ge 0$, and $w_{\text{borrow}} \le 0$, and the return on the borrowing risk-free asset is $r_b>0$.


In [25]:
# Case #1: multiple borrowing rates
raver = 3

# Risk-free saving
rs = 0.02
# Risk-free borrowing
rb = 0.04

In [26]:
# Convention that borrowing weight is positive (this is what is coded in portfolio class)
Q = np.zeros((n + 2, n + 2))
Q[2:, 2:] = raver * cov
Q = matrix(Q, tc="d")
p = np.array([-rs, rb] + list(-means))
p = matrix(p, (n + 2, 1), tc="d")
# Constraint: saving weight positive, borrowing weight positive
G = np.zeros((2, n + 2))
G[0, 0] = G[1, 1] = -1
G = matrix(G, (2, n+2), tc="d")
h = matrix([0, 0], (2, 1), tc="d")
# Constraint: fully-invested portfolio
A = matrix([1, -1] + n*[1], (1, n+2), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
if sol["status"] == "optimal":
    wgts_optimal = np.array(sol["x"]).flatten()[2:]
else:
    wgts_optimal = None
print('Optimal risky portfolio for investor with risk aversion of ' + f'{raver:.1f}' + ' is: ', wgts_optimal)
print('Saving weight: ' + f'{np.array(sol["x"]).flatten()[0]: .1%}')
print('Borrow weight: ' + f'{np.array(sol["x"]).flatten()[1]: .1%}')
print('Total weight in risky assets is: ' + f'{np.sum(wgts_optimal): .1%}')

     pcost       dcost       gap    pres   dres
 0: -3.2806e-02 -1.0394e-02  2e+00  1e+00  1e+00
 1: -1.5421e-02 -7.9172e-02  7e-02  3e-02  3e-02
 2: -2.6430e-02 -3.5970e-02  1e-02  2e-16  7e-18
 3: -3.5164e-02 -3.5483e-02  3e-04  2e-16  6e-18
 4: -3.5254e-02 -3.5258e-02  4e-06  2e-16  6e-18
 5: -3.5255e-02 -3.5255e-02  4e-08  6e-17  7e-18
Optimal solution found.
Optimal risky portfolio for investor with risk aversion of 3.0 is:  [0.17008091 0.20359394 0.24242445]
Saving weight:  38.4%
Borrow weight:  0.0%
Total weight in risky assets is:  61.6%


In [27]:
# Convention where borrowing portfolio weight is negative
Q = np.zeros((n + 2, n + 2))
Q[2:, 2:] = raver * cov
Q = matrix(Q, tc="d")
p = np.array([-rs, -rb] + list(-means))
p = matrix(p, (n + 2, 1), tc="d")
# Constraint: saving weight positive, borrowing weight negative
G = np.zeros((2, n + 2))
G[0, 0] = -1
G[1, 1] = 1
G = matrix(G, (2, n+2), tc="d")
h = matrix([0, 0], (2, 1), tc="d")
# Constraint: fully-invested portfolio
A = matrix(np.ones(n+2), (1, n+2), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
if sol["status"] == "optimal":
    wgts_optimal = np.array(sol["x"]).flatten()[2:]
else:
    wgts_optimal = None
print('Optimal risky portfolio for investor with risk aversion of ' + f'{raver:.1f}' + ' is: ', wgts_optimal)
print('Saving weight: ' + f'{np.array(sol["x"]).flatten()[0]: .1%}')
print('Borrow weight: ' + f'{np.array(sol["x"]).flatten()[1]: .1%}')
print('Total weight in risky assets is: ' + f'{np.sum(wgts_optimal): .1%}')

     pcost       dcost       gap    pres   dres
 0: -3.2806e-02 -1.0394e-02  2e+00  1e+00  1e+00
 1: -1.5421e-02 -7.9172e-02  7e-02  3e-02  3e-02
 2: -2.6430e-02 -3.5970e-02  1e-02  2e-16  7e-18
 3: -3.5164e-02 -3.5483e-02  3e-04  2e-16  6e-18
 4: -3.5254e-02 -3.5258e-02  4e-06  2e-16  6e-18
 5: -3.5255e-02 -3.5255e-02  4e-08  6e-17  7e-18
Optimal solution found.
Optimal risky portfolio for investor with risk aversion of 3.0 is:  [0.17008091 0.20359394 0.24242445]
Saving weight:  38.4%
Borrow weight: -0.0%
Total weight in risky assets is:  61.6%


#### Risky-assets only (without risk-free assets)

In [28]:
# Method #1: theoretical solution without short-sale constraint
w = np.linalg.solve(cov, np.ones(n))
GMV = w / np.sum(w)
w = np.linalg.solve(cov, means)
piMu = w / np.sum(w)
a = np.sum(w)
wgts_optimal = (a/raver)*piMu + (1-a/raver)*GMV
print('Optimal risky portfolio for investor with risk aversion of ' + f'{raver:.1f}' + ' is: ', wgts_optimal)

Optimal risky portfolio for investor with risk aversion of 3.0 is:  [0.47066983 0.36022009 0.16911008]


In [29]:
# Method #2: numerical solution without short-sale constraint
Q = matrix(raver * cov, tc="d")
p = matrix(-means, (n, 1), tc="d")
# Constraint: short-sales not allowed, saving weight positive, borrowing weight negative
G = matrix(-np.identity(n), tc="d")
h = matrix(np.zeros(n), (n, 1), tc="d")
# Constraint: fully-invested portfolio
A = matrix(np.ones(n), (1, n), tc="d")
b = matrix([1], (1, 1), tc="d")
sol = Solver(Q, p, G, h, A, b)
if sol["status"] == "optimal":
    wgts_optimal = np.array(sol["x"]).flatten()
else:
    wgts_optimal = None
print('Optimal risky portfolio for investor with risk aversion of ' + f'{raver:.1f}' + ' is: ', wgts_optimal)

     pcost       dcost       gap    pres   dres
 0: -3.0013e-02 -1.0336e+00  1e+00  0e+00  2e+00
 1: -3.0040e-02 -4.3508e-02  1e-02  1e-16  3e-02
 2: -3.0567e-02 -3.1779e-02  1e-03  1e-16  2e-03
 3: -3.0682e-02 -3.0757e-02  7e-05  1e-16  3e-05
 4: -3.0684e-02 -3.0685e-02  9e-07  8e-17  3e-07
 5: -3.0684e-02 -3.0684e-02  9e-09  8e-17  3e-09
Optimal solution found.
Optimal risky portfolio for investor with risk aversion of 3.0 is:  [0.47066932 0.36021959 0.16911109]
